## Create polyedral globe with 12 sides

### Load libraries

In [1]:
import arcpy
import numpy as np
import os
import glob
import tools
import face_definitions
import sys
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
from math import pi, sin, cos, tan
from collections import namedtuple


#### Set workspace


In [2]:
workspace = r"C:\Computation\CUNI\02_LS1\MATKARTO\testing_py_globes"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = 1
os.chdir(workspace)
# create processing folder in workspace
if not os.path.exists("processing"):
    os.mkdir("processing")
if not os.path.exists("check"):
    os.mkdir("check")

### Set global variables

In [3]:
scale = 10000000
R = 6378000
R_scaled = R/scale



#### Save all vector datasets to be shown on the globe to the data dir in the workspace

In [4]:
# load data in shapefile or geojson
data_shp = glob.glob(r"data/*.shp")
data_json = glob.glob(r"data/*.geojson")
data = data_shp + data_json
print(f'Vector datasets found: {data}')
side_len = 50
cir_rad = (side_len / 2) * 1.618

# Open ArcGIS Pro project
project_path = r"project\globes.aprx"
pro_project = arcpy.mp.ArcGISProject(project_path) # Creates a blank project

# Remove all layouts from project
for layout in pro_project.listLayouts():
    layout.removeElement(layout)

# remove all maps from project
maps = pro_project.listMaps()  # Get a list of maps
for map in maps: 
    pro_project.deleteItem(map)  
print(pro_project.listLayouts())
print(pro_project.listMaps())
# create A3 layout
layout = pro_project.createLayout(594, 420, 'MILLIMETER')


Vector datasets found: ['data\\continents.shp']
[]
[]


### Create 12 globe faces

In [5]:
# load faces
def shift_s(d):
    return sin(pi / 5) * d / tan(pi / 5)

def shift_s2(d):
    return sin(2*pi / 5) * d / tan(pi / 5)

def shift_c(d):
    return cos(pi / 5) * d / tan(pi / 5)

def shift_c2(d):
    return cos(2*pi / 5) * d / tan(pi / 5)

def cir_rad(d):
    R = d / (2 * sin(pi / 5))
    return R

def shift_d(d):
    return 2*(tan(0.3*pi) * (d / 2))

faces = face_definitions.face_data
# definision for northern hemisphere
shift_x = [0, 
            shift_s2(side_len),
            shift_s(side_len),
            -shift_s(side_len),
            -shift_s2(side_len),
            0, 
            0, 
            0, 
            0, 
            0, 
            0, 
            0]

shift_y = [
            -shift_d(side_len), 
            -shift_c2(side_len), 
            shift_c(side_len), 
            shift_c(side_len), 
            -shift_c2(side_len), 
            0,
            0, 
            0, 
            0, 
            0, 
            0, 
            0]

rotate = [0, 72, 144, -144, -72, 0, -72, -144, 144, 72, 0, 36]

# center for N Africa
val = faces['Face 1']
pentagram = tools.layout.pent_create(a = side_len, angle = val[10])
pentagram_shift = tools.layout.pent_move(pentagram, x_shift=val[8]+shift_x[0], y_shift=val[9]+shift_y[0]) # face 1
ca_c = tools.layout.pent_center(pentagram_shift)
ca_c_x, ca_c_y = ca_c[0], ca_c[1]

print(f'Center_x : {ca_c_x}, center_y: {ca_c_y}')
# update southern hemisphere
Shift = namedtuple("Shift", ["x_value", "x_index", "y_value", "y_index"])
shift_south = [
    Shift(ca_c_x + shift_s(side_len), 5, ca_c_y - shift_c(side_len), 5),
    Shift(ca_c_x + shift_s(side_len), 11, ca_c_y - shift_c(side_len)-shift_d(side_len), 11)
]
for shift in shift_south:
    if 0 <= shift.x_index < len(shift_x):
        shift_x[shift.x_index] = shift.x_value
    if 0 <= shift.y_index < len(shift_y):
        shift_y[shift.y_index] = shift.y_value

# Set center to antlantida
val = faces['Face 12']
pentagram = tools.layout.pent_create(a = side_len, angle = val[10])
pentagram_shift = tools.layout.pent_move(pentagram, x_shift=val[8]+shift_x[11], y_shift=val[9]+shift_y[11]) # face 1
ca_c = tools.layout.pent_center(pentagram_shift)    
ca_c_x, ca_c_y = ca_c[0], ca_c[1]
# shift for southern hemisphere update
Shift = namedtuple("Shift", ["x_value", "x_index", "y_value", "y_index"])
shift_south = [
    Shift(ca_c_x - shift_s(side_len), 8, ca_c_y - shift_c(side_len), 8),
    Shift(ca_c_x + shift_s(side_len), 7, ca_c_y - shift_c(side_len), 7),
    Shift(ca_c_x + shift_s2(side_len), 6, ca_c_y + shift_c2(side_len), 6),
    Shift(ca_c_x - shift_s2(side_len), 9, ca_c_y + shift_c2(side_len), 9),


]

for shift in shift_south:
    if 0 <= shift.x_index < len(shift_x):
        shift_x[shift.x_index] = shift.x_value
    if 0 <= shift.y_index < len(shift_y):
        shift_y[shift.y_index] = shift.y_value




i = 0
# iterrate over keys and values in faces
for k, v in faces.items():
    if i == 2 or i == 10 or i == 0 or i ==4 or i ==1 or i ==3 or i == 5 or i == 11 or i == 9 or i ==8 or i ==7 or i ==6:
        print(f'WORKING ON :{k}')
        # Create new map to project
        map = pro_project.createMap(f"Map_{i+1}")
        # create face's boundary
        # convert degrees to radians
        U_B = np.radians(v[0])
        V_B = np.radians(v[1])
        # uk = tools.projections.normalize_longitude(v[6])
        # vk = tools.projections.normalize_latitude(v[7])
        uk = v[6]
        vk = v[7]
        # create face's boundary
        uk_rad = np.radians(v[6])
        vk_rad = np.radians(v[7])
        # print inputs to boundary
        XB, YB = tools.projections.boundary(U_B, V_B, R=R, uk=uk_rad, vk=vk_rad)
        point_array = arcpy.Array([arcpy.Point(x, y) for x, y in zip(XB, YB)])
        print(f'Creating face: {k}, cartographic pole, u: {v[6]}, v: {v[7]}.')
        # Combine X and Y into a list of tuples
        boundary_coords = list(zip(XB, YB))
        # Create a SpatialReference object from the WKT string
        wkt = tools.projections.update_wkt_projection(new_lon=vk, new_lat=uk)
        sr = arcpy.SpatialReference(text=wkt)
        # define map spatial reference
        map.spatialReference = sr
        # Convert geodataframe to shapefile and apply projection from wkt
        boundary_poly = arcpy.Polygon(arcpy.Array([arcpy.Point(*coords) for coords in boundary_coords]))
        boundary_ap = arcpy.management.CreateFeatureclass(out_path="in_memory", out_name="boundary", geometry_type="POLYGON", spatial_reference=sr)
        desc = arcpy.Describe(boundary_ap)
        with arcpy.da.InsertCursor(boundary_ap, ["SHAPE@"]) as cursor:
            cursor.insertRow([boundary_poly])
        boundary_layer = arcpy.management.MakeFeatureLayer(boundary_ap, "boundary_layer")[0]
        # save boundary layer to temp 
        boundary_layer_path = os.path.join("processing", f"boundary_face_{i}.shp")
        arcpy.management.CopyFeatures(boundary_layer, boundary_layer_path)
        boundary_map = arcpy.management.MakeFeatureLayer(boundary_layer_path, "boundary_map")[0]
        desc = arcpy.Describe(boundary_map)
        # spatial_ref = desc.spatialReference
        # spatial_ref_wkt = spatial_ref.exportToString()
       
        # 1. layers to map
        map.rotation = 60  # Nastavení rotace mapy na 60 stupňů
        map.addLayer(boundary_map)
        boundary_layer = map.listLayers("boundary_map")[0]

        symbology = boundary_layer.symbology
        symbology.renderer.symbol.color = {'RGB': [40, 40, 175, 0]} 
        boundary_layer.symbology = symbology

        #2. pentagon
        # packing_north hemisphere
       
        
        print(f'Shift x: {shift_x}')
        print(f'Shift y: {shift_y}')
        pentagon = tools.layout.pent_create(a = side_len, angle = v[10])
        
        pentagon_shift = tools.layout.pent_move(pentagon, x_shift=v[8]+shift_x[i], y_shift=v[9]+shift_y[i])
        pent_center = tools.layout.pent_center(pentagon_shift)
        print(f'{k} has center at {pent_center}')
        frame = tools.layout.frame_points(pentagon_shift)
        
        # 3. map to frame
        extent = desc.extent
        projected_extent = extent.projectAs(sr)
        map_frame = layout.createMapFrame(frame, map)
        map_frame.map = map
        # map_frame.map.rotation = 60
        map_frame.map.spatialReference = sr
        map_frame.camera.setExtent(extent)
        map_frame.camera.heading = rotate[i]
        # map_frame.camera.rotation = rotation_angle 


    i +=1
pro_project.saveACopy('test/globes_test.aprx')
layout.exportToPDF("globe_faces.pdf")

"""
    # # Doesn't work
    # spatial_ref = arcpy.SpatialReference()
    # spatial_ref.loadFromString(wkt)
    # boundary_poly = arcpy.Polygon(point_array, spatial_ref)    
    # output_shapefile = os.path.join("check", f"boundary_polygon{i}.shp")
    # arcpy.CopyFeatures_management(boundary_poly, output_shapefile)
    # layout.exportToPDF("globe_face.pdf")
"""


Center_x : 150.0, center_y: 251.18090397644127
WORKING ON :Face 1
Creating face: Face 1, cartographic pole, u: 31.71745, v: 0.
Shift x: [0, 65.45084971874738, 40.45084971874738, -40.45084971874738, -65.45084971874738, 190.45084971874738, 255.90169943749476, 230.90169943749476, 150.0, 125.0, 0, 190.45084971874738]
Shift y: [-68.81909602355867, -21.266270208801004, 55.67581822058035, 55.67581822058035, -21.266270208801004, 195.50508575586093, 147.95225994110325, 71.01017151172192, 71.01017151172192, 147.95225994110325, 0, 126.68598973230226]
Face 1 has center at (150.0, 251.18090397644127)
WORKING ON :Face 2
Creating face: Face 2, cartographic pole, u: 31.71745, v: 72.
Shift x: [0, 65.45084971874738, 40.45084971874738, -40.45084971874738, -65.45084971874738, 190.45084971874738, 255.90169943749476, 230.90169943749476, 150.0, 125.0, 0, 190.45084971874738]
Shift y: [-68.81909602355867, -21.266270208801004, 55.67581822058035, 55.67581822058035, -21.266270208801004, 195.50508575586093, 147.95

'\n    # # Doesn\'t work\n    # spatial_ref = arcpy.SpatialReference()\n    # spatial_ref.loadFromString(wkt)\n    # boundary_poly = arcpy.Polygon(point_array, spatial_ref)    \n    # output_shapefile = os.path.join("check", f"boundary_polygon{i}.shp")\n    # arcpy.CopyFeatures_management(boundary_poly, output_shapefile)\n    # layout.exportToPDF("globe_face.pdf")\n'